1. a

In [9]:
import numpy as np
from scipy.integrate import nquad

def solve_part_a(A, w, L=5.0):
   
    N = A.shape[0]

    def integrand(*coords):
        v = np.array(coords)
        exponent = -0.5 * v @ (A @ v) + w @ v
        return np.exp(exponent)

    bounds = [(-L, L)] * N
    numeric_val, err = nquad(integrand, bounds)

    A_inv = np.linalg.inv(A)
    detA_inv = np.linalg.det(A_inv)
    closed_form = np.sqrt((2 * np.pi)**N * detA_inv) * \
                  np.exp(0.5 * (w @ (A_inv @ w)))

    return numeric_val, closed_form, err
if __name__ == "__main__":
    main()

=== 1D Example ===
Numeric integral = 2.275876 ± 1.07e-10
Closed-form     = 2.275876

=== 2D Example ===
Numeric integral = 4.415037 ± 1.44e-08
Closed-form     = 4.415037


1. b

In [13]:
import numpy as np
import scipy.integrate as spi

def is_spd(A):
    return np.allclose(A, A.T) and np.all(np.linalg.eigvals(A) > 0)

def integrand(*v, A, w):
    v = np.array(v, dtype=float)
    exponent = -0.5 * v @ (A @ v) + w @ v
    return np.exp(exponent)

def closed_form_integral(A, w):
    # If A isn't SPD, the formula doesn't apply (diverges or is invalid)
    if not is_spd(A):
        return float('nan')
    A_inv = np.linalg.inv(A)
    N = A.shape[0]
    detAinv = np.linalg.det(A_inv)
    factor = np.sqrt((2*np.pi)**N * detAinv)
    expo  = np.exp(0.5 * (w @ (A_inv @ w)))
    return factor * expo

def numeric_integral(A, w):
    if not is_spd(A):
        return (float('nan'), float('nan'))
    N = len(w)
    bounds = [(-10,10)] * N

    def wrapper(*v):
        return integrand(*v, A=A, w=w)

    val, err = spi.nquad(wrapper, bounds)
    return (val, err)


# Compare A vs. A'
def main():
    A = np.array([[4,2,1],
                  [2,5,3],
                  [1,3,6]], dtype=float)

    A_prime = np.array([[4,2,1],
                        [2,1,3],
                        [1,3,6]], dtype=float)

    w = np.array([1,2,3], dtype=float)

    print("Matrix A:")
    print(A)
    print("Is SPD?", is_spd(A))
    print("Closed-form integral:", closed_form_integral(A, w))
    num_val_A, num_err_A = numeric_integral(A, w)
    print(f"Numeric integral = {num_val_A} ± {num_err_A}")

    print("\nMatrix A':")
    print(A_prime)
    print("Is SPD?", is_spd(A_prime))
    print("Closed-form integral:", closed_form_integral(A_prime, w))
    num_val_Ap, num_err_Ap = numeric_integral(A_prime, w)
    print(f"Numeric integral = {num_val_Ap} ± {num_err_Ap}")

print("Discussion:")
print("Matrix A is symmetric positive definite (SPD), so the integral is well-defined.")
print("A' differs in the (2,2) entry, and likely isn't SPD. If not SPD, the integral may diverge")
print("or the closed-form formula does not apply. Hence the difference in results for A vs. A'.")
if __name__ == "__main__":
    main()

Discussion:
Matrix A is symmetric positive definite (SPD), so the integral is well-defined.
A' differs in the (2,2) entry, and likely isn't SPD. If not SPD, the integral may diverge
or the closed-form formula does not apply. Hence the difference in results for A vs. A'.
Matrix A:
[[4. 2. 1.]
 [2. 5. 3.]
 [1. 3. 6.]]
Is SPD? True
Closed-form integral: 4.275823659011514
Numeric integral = 4.275823659012836 ± 2.5251607885961626e-08

Matrix A':
[[4. 2. 1.]
 [2. 1. 3.]
 [1. 3. 6.]]
Is SPD? False
Closed-form integral: nan
Numeric integral = nan ± nan


1. c

In [14]:
import numpy as np
from scipy.integrate import nquad

def is_spd(A, tol=1e-12):
    """Check if A is symmetric positive-definite."""
    if not np.allclose(A, A.T, atol=tol):
        return False
    eigenvals = np.linalg.eigvals(A)
    return np.all(eigenvals > 0)

def partition_function(A, w, L=5.0):
   
    N = len(w)
    def integrand(*coords):
        v = np.array(coords)
        exponent = -0.5 * v @ (A @ v) + w @ v
        return np.exp(exponent)

    bounds = [(-L, L)] * N
    val, err = nquad(integrand, bounds)
    return val, err

def moment_numeric(A, w, powers, L=5.0):
  
    # 1) compute partition function Z
    Z, _ = partition_function(A, w, L)

    # 2) compute numerator = ∫ (v^powers) exp(...) dv
    N = len(w)
    def integrand_m(*coords):
        v = np.array(coords)
        exponent = -0.5 * v @ (A @ v) + w @ v
        # product of v[i]^powers[i]
        prod = 1.0
        for i, p in enumerate(powers):
            prod *= v[i]**p
        return prod * np.exp(exponent)

    bounds = [(-L, L)] * N
    val, _ = nquad(integrand_m, bounds)
    return val / Z

def moment_closed_form(A, w, powers):
    if not is_spd(A):
        return float('nan')

    A_inv = np.linalg.inv(A)
    mu = A_inv @ w
    total_power = sum(powers)

    if total_power == 1:
        # e.g. (1,0,0) => <v1> = mu[0]
        i = np.argmax(powers)
        return mu[i]

    elif total_power == 2:
        # e.g. (2,0,0) => <v1^2>, or (1,1,0) => <v1 v2>
        idxs = []
        for i, p in enumerate(powers):
            idxs.extend([i]*p)  # e.g. p=2 => i,i
        i, j = idxs
        return mu[i]*mu[j] + A_inv[i,j]

    else:
        return float('nan')  # need Wick's theorem for higher-order

def main():
    A = np.array([
        [4,2,1],
        [2,5,3],
        [1,3,6]
    ], dtype=float)
    w = np.array([1,2,3], dtype=float)

   
    moment_list = [
        ("v1",   (1,0,0)),
        ("v2",   (0,1,0)),
        ("v3",   (0,0,1)),
        ("v1^2", (2,0,0)),
        ("v2^2", (0,2,0)),
        ("v3^2", (0,0,2)),
        ("v1v2", (1,1,0)),
        ("v2v3", (0,1,1)),
        ("v1v3", (1,0,1)),
    ]

    print("Part (c): Correlation Functions and Moments\n")
    print("Matrix A:\n", A)
    print("Vector w:\n", w)
    print("\nMoments (Numeric vs. Closed-form):")

    for label, powers in moment_list:
        val_num = moment_numeric(A, w, powers, L=6.0)
        val_cf  = moment_closed_form(A, w, powers)
        print(f"{label}: numeric={val_num:.6f}, closed-form={val_cf:.6f}")

    print("\n(Any mismatch is due to numerical integration error or if sum(powers)>2, we used a simple formula.)")

if __name__ == "__main__":
    main()

Part (c): Correlation Functions and Moments

Matrix A:
 [[4. 2. 1.]
 [2. 5. 3.]
 [1. 3. 6.]]
Vector w:
 [1. 2. 3.]

Moments (Numeric vs. Closed-form):
v1: numeric=0.089552, closed-form=0.089552
v2: numeric=0.104478, closed-form=0.104478
v3: numeric=0.432836, closed-form=0.432836
v1^2: numeric=0.321452, closed-form=0.321452
v2^2: numeric=0.354199, closed-form=0.354199
v3^2: numeric=0.426153, closed-form=0.426153
v1v2: numeric=-0.124972, closed-form=-0.124972
v2v3: numeric=-0.104032, closed-form=-0.104032
v1v3: numeric=0.053687, closed-form=0.053687

(Any mismatch is due to numerical integration error or if sum(powers)>2, we used a simple formula.)
